IDMB movie review dataset 

# Import libraries

In [1]:
import glob
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
def make_data_frame(files, sentiment):
    df = pd.DataFrame(columns=['review', 'sentiment'])
    for file in positive_review_train_files:
        with open(file,'r', encoding='utf-8') as infile:
            txt = infile.read()
            df = df.append({'review': txt, 'sentiment': sentiment }, ignore_index=True)
    return df
  
    

Taking review training files

In [3]:
positive_review_train_files=glob.glob("/Users/luo/IDMB/aclImdb/train/pos/*.txt")
len(positive_review_train_files)

12500

In [4]:
df_pos_train = make_data_frame(positive_review_train_files, 1)


In [5]:
df_pos_train.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [6]:
negative_review_train_files =glob.glob("/Users/luo/IDMB/aclImdb/train/neg/*.txt")
len(negative_review_train_files)

12500

In [7]:
df_neg_train = make_data_frame(negative_review_train_files, 0)


In [8]:
df_neg_train.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,0
1,Bizarre horror movie filled with famous faces ...,0
2,"A solid, if unremarkable film. Matthau, as Ein...",0
3,It's a strange feeling to sit alone in a theat...,0
4,"You probably all already know this by now, but...",0


In [9]:
df_train = df_pos_train.append(df_neg_train, ignore_index=True)

In [10]:
df_train.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [11]:
df_train.tail()

,review,sentiment
24995,About a year ago I finally gave up on American...,0
24996,When I saw the elaborate DVD box for this and ...,0
24997,"Last November, I had a chance to see this film...",0
24998,Great movie -I loved it. Great editing and use...,0
24999,"Enchanted April is a tone poem, an impressioni...",0


Taking review testing files

In [12]:
positive_review_test_files=glob.glob("/Users/luo/IDMB/aclImdb/test/pos/*.txt")
len(positive_review_test_files)

12500

In [13]:
df_pos_test = make_data_frame(positive_review_test_files, 1)


In [14]:
df_pos_test.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [15]:
negative_review_test_files=glob.glob("/Users/luo/IDMB/aclImdb/test/neg/*.txt")
len(negative_review_test_files)

12500

In [16]:
df_neg_test = make_data_frame(negative_review_test_files, 0)

In [17]:
df_neg_test.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,0
1,Bizarre horror movie filled with famous faces ...,0
2,"A solid, if unremarkable film. Matthau, as Ein...",0
3,It's a strange feeling to sit alone in a theat...,0
4,"You probably all already know this by now, but...",0


In [18]:
df_test = df_pos_test.append(df_neg_test, ignore_index=True)

# Preprocessing

In [19]:
reviews_df = df_train.append(df_test, ignore_index=True)


len(reviews_df)

50000

In [20]:
reviews_df.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [21]:
#reviews_df.isnull().values.any()


#convert the review data frames into lists 
reviews_list = reviews_df['review'].values.tolist()
for i in  range(0,10):
    print("review ", i , "\n")
    print(reviews_list[i])


review  0 

For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.
review  1 

Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV's "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina's pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wi

In [22]:
stop_words = set(stopwords.words('english'))  # take stopwors from nltk stopwords

In [23]:
clean_lines = []

In [24]:
for line in reviews_list:
    token = word_tokenize(line)    #tokenize the line 
    lower_token = [w.lower() for w in token]  #converting lower case
    table =str.maketrans('', '', string.punctuation)  
    stripped = [w.translate(table) for w in lower_token]  #removeing lower case
    words = [word for  word in stripped if word.isalpha()]  #removing numbers 
    
    words = [w for w in words if not w in stop_words] #removing stopwords
    clean_lines.append(words)
    

In [25]:
len(clean_lines)

50000

# Train word2vec Embedding

In [26]:
import gensim
from gensim.models import Word2Vec

In [27]:
EMBADING_DIM = 100

In [28]:
model = Word2Vec(sentences=clean_lines,  size = EMBADING_DIM, workers = 6, window=5, min_count=1)

In [29]:
word = list(model.wv.vocab)

In [30]:
print(len(word))

65370


In [31]:
model.wv.most_similar("please")

[('email', 0.6666344404220581),
 ('u', 0.6330054402351379),
 ('website', 0.6317614316940308),
 ('someday', 0.6265828609466553),
 ('want', 0.6161319613456726),
 ('download', 0.6046980619430542),
 ('hopefully', 0.5969879627227783),
 ('ps', 0.5965090990066528),
 ('purchase', 0.5916213989257812),
 ('nonuk', 0.5842483639717102)]

In [32]:
file = "idmb_embadading_word2vec.txt"
model.wv.save_word2vec_format(file, binary=False)

# Train Fasttest (Facebook) Embedding

In [35]:
import gensim
from gensim.models import FastText

In [36]:
EMBADING_DIM = 100

In [37]:
model_ted = FastText(sentences=clean_lines,  size = EMBADING_DIM, workers = 6, window=5, min_count=1)

In [38]:
word_firstt = list(model_ted.wv.vocab)

NameError: name 'model_fasttext' is not defined

In [ ]:
model_fasttext.wv.most_similar("please")


# Classfication using Pretrained Embadding model

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
df_test.head()
# len(df_test)

In [ ]:
df_train.head()


In [ ]:
df = df_train.append(df_test ,ignore_index=True)
reviews = df['review'].values
# len(reviews)

In [ ]:
for i in range(0, len(reviews)):
    token = word_tokenize(reviews[i])
    lower_token = [w.lower() for w in token]
    table =str.maketrans('', '', string.punctuation)  
    stripped = [w.translate(table) for w in lower_token]  #removeing lower case
    words = [word for  word in stripped if word.isalpha()]  #removing numbers 
    words = [w for w in words if not w in stop_words] #removing stopwords
    sentence = ' '.join(word for word in words)
    reviews[i]=sentence


In [ ]:
tokenizer_obj = Tokenizer()

In [ ]:
tokenizer_obj.fit_on_texts(reviews)

In [ ]:
sequences = tokenizer_obj.texts_to_sequences(reviews)

In [ ]:
word_index = tokenizer_obj.word_index

In [ ]:
max_len = max([len(lenght.split()) for lenght in reviews])
print(max_len)

In [ ]:
sequences_pad = pad_sequences(sequences, maxlen=max_len)

In [ ]:
sequences_pad.shape